# Utilisation de VGG-16 pré-entraîné

Ressource 2.5 -https://openclassrooms.com/fr/courses/4470531-classez-et-segmentez-des-donnees-visuelles/5097666-tp-implementez-votre-premier-reseau-de-neurones-avec-keras

In [27]:
# On importe et charge un modèle pré-entrainé : VGG-16

from keras.applications.vgg16 import VGG16

model = VGG16() # Création du modèle VGG-16 implementé par Keras

In [28]:
# Là on vérifie juste que VGG prédit bien. On teste avec une img, mais il faut la préprocesser avant de
# pouvoir la tester dans le modèle

from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

img = load_img('../data/poule1.jpeg', target_size=(224, 224))  # Charge l'image et la redimensionne

img = img_to_array(img)  # Keras traite les images comme des tableaux numpy : convertit l'image chargée
# en tableau numpy

# Le réseau doit recevoir en entrée une collection d'images, stockée dans un tableau de 4 dimensions,
# où les dimensions correspondent dans l'ordre à (nombre d'images, largeur, hauteur, profondeur).
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
# reshape ajoute la première dimension (nombre d'images = 1) à notre image

img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16

In [29]:
y = model.predict(img)  # Prédir la classe de l'image (parmi les 1000 classes d'ImageNet)
# ca renvoit une liste de 1000 proba (une par classe/output)

In [30]:
from keras.applications.vgg16 import decode_predictions

# Afficher les 3 classes les plus probables
print('Top 3 :', decode_predictions(y, top=3)[0])

Top 3 : [('n01514859', 'hen', 0.9546121), ('n01514668', 'cock', 0.0442258), ('n01807496', 'partridge', 0.00086757954)]


In [6]:
# Le modèle est sur à 95% que c'est une poule et 4% que c'est un coq.

In [11]:
# Test avec une image de cochon d'Inde
img1 = load_img('../data/cochon_dinde.jpg', target_size=(224,224))
img1 = img_to_array(img1)

img1 = img1.reshape((1,img1.shape[0],img1.shape[1],img1.shape[2]))
img1 = preprocess_input(img1)

y1 = model.predict(img1)
decode_predictions(y1,top=3)[0]

[('n02364673', 'guinea_pig', 0.99892503),
 ('n02342885', 'hamster', 0.0010561245),
 ('n07714990', 'broccoli', 1.0151749e-05)]

# Dataset

Ressource 2.6 : https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

## On charge les données

In [8]:
# On importe la classe qu'on veut. Elle permet de créer pleins de
# variants d'une même image - intéressant quand on a peu d'images
from keras.preprocessing.image import ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator() # on peut lui proposer pleins
# de paramètres sur les variants qu'on souhaite obtenir

In [50]:
# On charge les données qui sont dans le ML5/data folder grâce à flow_from_directory

train_generator = train_datagen.flow_from_directory(
    directory=r"../data/train/", # chemin du fichier
    target_size=(224, 224), #  size of your input images, every image will be resized to this size
    color_mode="rgb", # if B&W set "grayscale". if 3 color channels, "rgb"
    batch_size=128, # number of images to be yielded from the generator per batch
    class_mode="categorical", # if 2 classes to predict, "binary". if more, "categorical"
    shuffle=True, # shuffle the order of the image being yielded
    seed=42 # random seed
)

Found 6 images belonging to 2 classes.


In [51]:
valid_datagen = ImageDataGenerator()
valid_generator = valid_datagen.flow_from_directory(
    directory=r"../data/valid/", # chemin du fichier
    target_size=(224, 224), #  size of your input images, every image will be resized to this size
    color_mode="rgb", # if B&W set "grayscale". if 3 color channels, "rgb"
    batch_size=128, # number of images to be yielded from the generator per batch
    class_mode="categorical", # if 2 classes to predict, "binary". if more, "categorical"
    shuffle=True, # shuffle the order of the image being yielded
    seed=42 # random seed
)

Found 6 images belonging to 2 classes.


In [52]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    directory=r"../data/test/", # chemin du fichier
    target_size=(224, 224), #  size of your input images, every image will be resized to this size
    color_mode="rgb", # if B&W set "grayscale". if 3 color channels, "rgb"
    batch_size=128, # number of images to be yielded from the generator per batch
    class_mode="categorical", # if 2 classes to predict, "binary". if more, "categorical"
    shuffle=True, # shuffle the order of the image being yielded
    seed=42 # random seed
)

Found 6 images belonging to 1 classes.


## On entraîne et évalue le modèle, puis on prédit

In [74]:
# Création du modèle
from glob import glob
from keras.layers import Flatten, Dense
from keras import Model

src_path_train = '../data/train'
src_path_valid = '../data/valid'
src_path_test = '../data/test'
IMSIZE = [224, 224]

NBCLASSES = 2
train_image_files = glob(src_path_train + '/*/*.jp*g')
test_image_files = glob(src_path_test + '/*/*.jp*g')
 
def create_model():
    vgg = VGG16(input_shape=IMSIZE + [3], weights='imagenet', include_top=False)
 
    # Freeze existing VGG already trained weights
    for layer in vgg.layers:
        layer.trainable = False
     
    # get the VGG output
    out = vgg.output
     
    # Add new dense layer at the end
    x = Flatten()(out)
    x = Dense(NBCLASSES, activation='sigmoid')(x)
     
    model = Model(inputs=vgg.input, outputs=x)
     
    model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])
     
    model.summary()
     
    return model
 
mymodel = create_model()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [75]:
from keras.callbacks import EarlyStopping

epochs = 30
batch_size=32

early_stop = EarlyStopping(monitor='val_loss',patience=2)
 
mymodel.fit(
  train_generator,
  validation_data=valid_generator,
  epochs=epochs,
  # steps_per_epoch= len(train_generator) // batch_size,
  # validation_steps=len(test_image_files) // batch_size,
  callbacks=[early_stop]
)

Epoch 1/30
1/1 [==============================] - 4s 4s/step - loss: 2.3645 - accuracy: 0.8333 - val_loss: 10.3266 - val_accuracy: 0.5000
Epoch 2/30
1/1 [==============================] - 3s 3s/step - loss: 6.9786 - accuracy: 0.8333 - val_loss: 9.2184 - val_accuracy: 0.5000
Epoch 3/30
1/1 [==============================] - 3s 3s/step - loss: 1.7797e-13 - accuracy: 1.0000 - val_loss: 9.8160 - val_accuracy: 0.5000
Epoch 4/30
1/1 [==============================] - 3s 3s/step - loss: 3.8542e-18 - accuracy: 1.0000 - val_loss: 10.9951 - val_accuracy: 0.5000


In [ ]:
# STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

# STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

# model.compile(optimizer="Adam", loss="mse", metrics=["mae"], run_eagerly=True)
# model.fit(train_generator,
                    # steps_per_epoch=STEP_SIZE_TRAIN,
                    # validation_data=valid_generator,
                    # validation_steps=STEP_SIZE_VALID,
#                     epochs=10
# )

In [76]:
score = mymodel.evaluate(test_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1/1 [==============================] - 1s 1s/step - loss: 8.5819 - accuracy: 0.8333
Test loss: 8.581878662109375
Test accuracy: 0.8333333134651184


In [80]:
### PREDICTIONS DU MODELE VGG PRE-ENTRAINE ###

img2 = load_img('../data/test/Test_folder/chat2.jpeg', target_size=(224,224))
img2 = img_to_array(img2)

img2 = img2.reshape((1,img2.shape[0],img2.shape[1],img2.shape[2]))
img2 = preprocess_input(img2)

y2 = model.predict(img2)
decode_predictions(y2,top=3)[0]

[('n02123045', 'tabby', 0.18181096),
 ('n02124075', 'Egyptian_cat', 0.17472121),
 ('n02123159', 'tiger_cat', 0.10490228)]

In [89]:
### PREDICTION DU MODELE VGG AVEC TRANSFER LEARNING ###

img2 = load_img('../data/test/Test_folder/chat2.jpeg', target_size=(224,224))
img2 = img_to_array(img2)

img2 = img2.reshape((1,img2.shape[0],img2.shape[1],img2.shape[2]))
img2 = preprocess_input(img2)

y2 = mymodel.predict(img2)
y2

array([[1.00000e+00, 9.41952e-27]], dtype=float32)

Ressources supplémentaire : https://www.tensorflow.org/tutorials/images/transfer_learning